<a href="https://colab.research.google.com/github/muy31/whisper-speech-sample/blob/main/WhisperDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<img src="https://blog.fastforwardlabs.com/images/cloudera-fast-forward-logo.png" width=300>

# Make your own recordings and transcriptions with OpenAI's Whisper!
_a fun diversion brought to you by [Melanie](https://www.linkedin.com/in/melanierbeck/), ML Research Manager at Cloudera Fast Forward Labs_


[OpenAI](https://openai.com/) [recently released](https://openai.com/blog/whisper/) Whisper, an automatic speech recognition (ASR) system that was trained on a colossal heap of audio data collected from the web.

## What makes Whisper unique?
Speech-to-text technology isn't new but Whisper might usher in the  next-generation of ASR systems in terms of the quality and capabilities delivered by a single model (rather than a collection of models, as most ASR systems are today.)  So what makes Whisper special?

1. It's "weakly supervised," meaning that it was trained on (audio, transcript) pairs wherein the transcriptions were _not_ human-validated, a typical hallmark of gold-stardard supervised audio datasets.  
2. This allows Whisper to be trained on a MASSIVE amount of data scraped from the web (680K hours) -- orders of magnitude larger than most audio datasets (5-30K hours)
3. The data contains audio snippets in dozens of languages allowing Whisper to be natively multilingual.
4. Whisper is also multitask -- it can perform transcription, translation, voice detection, and language detection.

You can learn more about Whisper in OpenAI's [recent post]((https://openai.com/blog/whisper/)), their [paper](https://cdn.openai.com/papers/whisper.pdf), and their open-source [codebase](https://github.com/openai/whisper). In fact, this notebook made use of OpenAI's [LibriSpeech](https://colab.research.google.com/github/openai/whisper/blob/master/notebooks/LibriSpeech.ipynb) Colab example as a starting point, so you should check that out as well!


This notebook allows you to play with Whisper by first creating your own audio recording, processing the audio into a format the model will understand, and feeding it into Whisper for translation or transcription! Let's get started.




## Installs and imports
The commands below will install the Python packages needed to record audio snippets and use Whisper models for speech-to-text transcription.

In [1]:
! pip install git+https://github.com/openai/whisper.git
! pip install sounddevice wavio
! pip install ipywebrtc notebook

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-0xqv_vk4
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-0xqv_vk4
  Resolved https://github.com/openai/whisper.git to commit dd985ac4b90cafeef8712f2998d62c59c3e62d22
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━

We also need the following in order to record audio from this notebook and process the resulting files.

In [2]:
!apt install ffmpeg
!apt-get install libportaudio2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 65.3 kB of archives.
After this operation, 223 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Fetched 65.3 kB in 1s (129 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 126109 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1.1) ...
Setting up libportaudio2:amd64 (19.6.0-1.1) ...
Processing triggers for 

In [3]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import torchaudio

from ipywebrtc import AudioRecorder, CameraStream
from IPython.display import Audio, display
import ipywidgets as widgets

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

## Make your recording

First, we need to enable some Colab widgets so that we can make an audio recording.

In [4]:
from google.colab import output
output.enable_custom_widget_manager()

### Time to record!

Press the circle button and start speaking. It may not look it, but the widget will be capturing sound. Click the circle button again when you are finished. The widget will immediately begin to play back what it captured.

In [5]:
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

The audio format captured above is not readable by PyTorch. In this step, we convert our recording into a format that PyTorch can understand.

In [16]:
with open('recording.webm', 'wb') as f:
    f.write(recorder.audio.value)
!ffmpeg -i recording.webm -ac 1 -f wav my_recording.wav -y -hide_banner -loglevel panic

### Alternatively...
If you don't want to make your own recording, you can instead upload an audio file to this notebook.


## Select options

Whisper is capable of performing transcriptions for many languages (though it performs better for some languages and worse for others.)

Whisper is also capable of detecting the input language. However, to be on the safe side, we can explicitly tell Whisper which language to expect.

In [7]:
language_options = whisper.tokenizer.TO_LANGUAGE_CODE
language_list = list(language_options.keys())

In [8]:
lang_dropdown = widgets.Dropdown(options=language_list, value='english')
output = widgets.Output()
display(lang_dropdown)

Dropdown(options=('english', 'chinese', 'german', 'spanish', 'russian', 'korean', 'french', 'japanese', 'portu…

Whisper is also capable of several tasks, including English-only transcription, Any-to-English translation, and non-English transcription.

Below you can select either "transcription" (which will yield text in the same language as the input language) or "translation" (which will transcribe from non-English to English).

![Whisper capabilities](https://cdn.openai.com/whisper/draft-20220920a/asr-training-data-desktop.svg)

Image from [Introducing Whisper](https://openai.com/blog/whisper/) by OpenAI

In [9]:
task_dropdown = widgets.Dropdown(options=['transcribe', 'translate'], value='transcribe')
output = widgets.Output()
display(task_dropdown)

Dropdown(options=('transcribe', 'translate'), value='transcribe')

## Load Whisper model

Whisper comes in five model sizes, four of which also have an optimized English-only version. This notebook loads "base"-sized models (bigger than "tiny" but smaller than the others), which require about 1GB of RAM.

If you selected English above, the cell below will load the optimized English-only version. Otherwise, it will load the multilingual model.


In [13]:
if lang_dropdown.value == "english":
  model = whisper.load_model("base.en")
else:
  model = whisper.load_model("base")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

100%|███████████████████████████████████████| 139M/139M [00:08<00:00, 17.7MiB/s]


Model is multilingual and has 71,825,920 parameters.


Finally, let's set the rest of our task and language options below and see what we've got. Check that your task and language settings are correct, but don't worry about the other defaults.

In [14]:
options = whisper.DecodingOptions(language=lang_dropdown.value, task=task_dropdown.value, without_timestamps=True)
options

DecodingOptions(task='transcribe', language='yoruba', temperature=0.0, sample_len=None, best_of=None, beam_size=None, patience=None, length_penalty=None, prompt=None, prefix=None, suppress_tokens='-1', suppress_blank=True, without_timestamps=True, max_initial_timestamp=1.0, fp16=True)

## Take Whisper for a test drive

All that's left to do now is feed our audio into Whisper.

The cell below performs the last processing steps to make this happen. First, it loads our PyTorch-ready audio file. Then it pads the audio into 30 sec segments. It creates a log-mel spectrogram of the audio and this is fed into Whisper along with the options we set above.



_Note: if you chose to upload your own audio file rather than create one through this notebook, you'll need to update the audio filename below._

In [19]:
audio = whisper.load_audio("my_recording.wav")
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).to(model.device)
result = model.decode(mel, options)

In [21]:
result.text

'Oshu keta odu'

### How well did Whisper do??

I read aloud the snippet above from one of my favorite books. For the record, Whisper is pretty dang close, making only two small mistakes -- both on proper names.   